In [ ]:
%matplotlib inline

In [ ]:
import lightgbm
import numpy as np
import pandas as pd
import plotnine as pln
import scipy
import statsmodels
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from skopt.searchcv import BayesSearchCV
from skopt.space import Categorical, Integer, Real
from tensorflow import keras

## Load the data

The data comes from [kaggle](https://www.kaggle.com/datasets/pablohfreitas/all-premier-league-matches-20102021).

In [ ]:
#  not sure if that will be necessary, but just in case
features_dictionary = (
    pd.read_csv(
        "../data/input/data_dictionary.txt", sep=":", skipinitialspace=True, header=None
    )
    .applymap(lambda x: x.strip())
    .set_index(0)
    .to_dict()[1]
)

In [ ]:
premiership_data = (
    pd.read_csv(
        "../data/input/df_full_premierleague.csv", index_col=0, parse_dates=["date"]
    )
    .drop(columns=["link_match"])  # unnecessary
    .sort_values(["season", "date"])
)

In [ ]:
premiership_data.head()

In [ ]:
premiership_data.shape

## Data pre-processing

Since we are looking for predictions for **future** matches, we need to stick only to variables that relate to the past performances of a given team. Hence, we will not be using features that apply to the current match. 

In [ ]:
X_fields = [
    k for k, v in features_dictionary.items() if "accumulated until the last match" in v
]
technical_fields = [
    "season",
    "date",
    "home_team",
    "away_team",
    "result_full",
    "result_ht",
]

In [ ]:
premiership_data = pd.concat(
    [
        premiership_data[technical_fields].reset_index(drop=True),
        premiership_data[X_fields].reset_index(drop=True),
    ],
    axis=1,
)

Now, we want to make sure that we will be using only the data that has ALL the features that we want to utilize. The features we are oriented at are aggregations and there is no value for them if a team plays their first home or away match during given season. So we will discard all the matches where this data is unavailable. In most cases, those will two first matches during the season (one home and one away). On rare occasions, it may happen that more matches will be discarded, since it is theoretically possible that the first two (or more) games of a given team were home- or away-only matches.

In [ ]:
premiership_data = premiership_data.dropna(how="any", subset=X_fields)

To check that, we can calculate the number of matches played by a team during the season. Premiership consists of 38 game series (also called matchweeks) so, if no hiccups were experienced during the season, a team should occur 36 times in our dataset (38 - first home game - first away game).

In [ ]:
num_games = pd.concat(
    [
        premiership_data.groupby(["season", "home_team"])["result_full"]
        .count()
        .reset_index()
        .rename(columns={"home_team": "team"}),
        premiership_data.groupby(["season", "away_team"])["result_full"]
        .count()
        .reset_index()
        .rename(columns={"away_team": "team"}),
    ],
    axis=0,
).rename(columns={"result_full": "games_played"})
num_games = num_games.groupby(["season", "team"])["games_played"].sum()

In [ ]:
plot_data = num_games.groupby("season").value_counts().rename("count").reset_index()

(
    pln.ggplot(data=plot_data, mapping=pln.aes(x="games_played", y="count"))
    + pln.geom_bar(stat="identity")
    + pln.facet_wrap("~season")
    + pln.labs(
        x="Games played (in the dataset)",
        y="",
        title="Number of games in the dataset per season played by given team",
    )
    + pln.theme_bw()
    + pln.theme(figure_size=(12, 10))
)

We see that in majority of cases, the teams in the dataset indeed played 36 games (so all that we could expect, given that two games of each team needed to be discarded due to lack of data). But at times it happens that there are less than 36 games in the dataset. For instance, in season 16/17, Chelsea and Leicester City have only 35 records. Leicester City played an away fixture vs. Liverppol in the 4th series, but Liverpool played only away games up to this point, so there have no record of home performance up to this point. Chelsea faced Burnley at home in the 3rd series, but Burnley played only away games up to this moment. Hence no away indicators for them.

The interesting thing is with season 20/21. There are (comparably) very few games played by all the teams. It's because the data for this season ends in March, whereas the season's last matchweek occurred on the 23rd of May '21, so we see why the data is missing. This, however, shouldn't jeopardize the analysis, so we will not remove this incomplete season.

TODO: Scrap the rest of the 20/21 season and potentially even 21/22.

Having removed the rows that are not useful in the analyses, we may proceed with the creation of our target variable.

In [ ]:
premiership_data = pd.concat(
    [
        premiership_data,
        premiership_data["result_full"]
        .str.split("-")
        .apply(
            lambda x: pd.Series(
                {
                    "goals_H": int(x[0]),  # number of goals scored by home team
                    "goals_A": int(x[1]),  # number of goals scored by away team
                    "win_H": int(x[0]) > int(x[1]),  # check if home team won
                    "win_A": int(x[0]) < int(x[1]),  # check if away team won
                }
            )
        ),
    ],
    axis=1,
)

In [ ]:
premiership_data["target_H"] = np.logical_and(
    premiership_data["goals_H"] >= 2, premiership_data["win_H"]
)
premiership_data["target_A"] = np.logical_and(
    premiership_data["goals_A"] >= 2, premiership_data["win_A"]
)

We have now two columns: `target_H` and `target_A` that store binary information whether home or away team meets the requirement of winning a game and scoring at least two goals (both of the conditions need to be met). What we should do now, is "flatten" the dataset so that we have one column - `target` that will contain both of those. We will repeat all the other fields and stack the dataset with itself. Each match will have then 2 rows - one for home and one for away team. Algorithm will see hence the same data twice during the learning process, but once it will be associated with the home team `target` and once with away team result. It will happen that for the same game both values will be 0s, but it is expected in such situation.

In [ ]:
training_data = (
    pd.concat(
        [
            pd.concat(
                [
                    premiership_data[technical_fields],
                    premiership_data[X_fields],
                    premiership_data["target_H"],
                ],
                axis=1,
            ).rename(columns={"target_H": "target"}),
            pd.concat(
                [
                    premiership_data[technical_fields],
                    premiership_data[X_fields],
                    premiership_data["target_A"],
                ],
                axis=1,
            ).rename(columns={"target_A": "target"}),
        ]
    )
    .reset_index(drop=True)
    .astype({"target": int})
)  # convert target to integer (0 vs 1)

### Split the data into training and validation datasets

We want to have a portion of the dataset left as validation dataset - to make final estimation of the performance quality of the algorithms we will develop. 
There are two splits we could apply:
- Stratification within season. Under this approach, we would sample a set of games from each season and leave them as the validation set. That would make sense, since during one season the teams participating in the league are constant, so our models would probably be more capable of making viable predictions.
- Stratification by season. In this setting, we will leave the freshest season (20/21) as the validation set. Training the model under previous assumption would be OK for discovery purposes - i.e. what are the indicators that a team will win. But this is kind of an exploratory analysis rather than forecasting. And we could argue that the latter is of more importance, since, for instance, we could use that to earn money, for example making bets (although it is morally-thin reason...).

That is why we opt for the second option and we will be using the data up to the season 20/21 as the training set and 20/21 games as the validation set. We could potentially extend the validation data set to seasons 19/20 and 20/21, but we will limit ourselves only to the last season to make the validation set coherent.

In [ ]:
training_data["train_val"] = np.where(
    training_data["season"] == "20/21", "val", "train"
)

In [ ]:
print(
    f"Validation set amounts to {(training_data['train_val']=='val').sum() / training_data.shape[0]:.2%} of the whole data set."
)

In [ ]:
X_train = training_data.loc[training_data["train_val"] == "train"][X_fields]
y_train = training_data.loc[training_data["train_val"] == "train"]["target"]

X_val = training_data.loc[training_data["train_val"] == "val"][X_fields]
y_val = training_data.loc[training_data["train_val"] == "val"]["target"]

## A bit of EDA

First, let's see if our datasets are balanced (or how much they are imbalanced, it's hard to expect that wins with two goals scored will as frequent as any other results).

In [ ]:
for subset, label in zip([y_train, y_val], ["training", "validation"]):
    g = (
        pln.ggplot(
            data=subset.astype("category").reset_index(), mapping=pln.aes(x="target")
        )
        + pln.geom_bar(stat="count", fill="#d1d1e0", color="black")
        + pln.geom_text(
            pln.aes(label=pln.after_stat("count")), stat="count", nudge_y=1, va="bottom"
        )
        + pln.labs(
            x="Target",
            y="",
            title=f"Counts of the TARGET variable for the {label} dataset",
        )
        + pln.theme_bw()
    )
    print(g)

Surprisingly enough, the imbalance level is not very serious. In the training set, around 30% of the games resulted in the win of one of the teams and that team scored at least 2 goals. In the validation set, this ratio is closer to 25%, but still - we are in the same ballpark - the differences between training and validation datasets are negligible. 

Yet another good news is that we do not have to resort to any special procedure of "fixing" extremely imbalanced dataset. Ratio of 1:3 is acceptable and will entail only the change in the metrics - we cannot use metrics that work well for balanced datasets (accuracy). Therefore we will be measuring performance with either F1 score or ROC AUC that are imbalance-resistant.

---

Having the dataset prepared, we can draw some plots to see what features contribute towards the target (winning, but scoring two goals at the minimum).

In [ ]:
plot_data = pd.concat([X_train, y_train.astype("category")], axis=1).melt(
    id_vars="target"
)
(
    pln.ggplot(pln.aes(x="target", y="value"), plot_data)
    + pln.geom_boxplot(fill="#d1d1e0", color="black")
    + pln.facet_wrap("~variable", scales="free_y")
    + pln.labs(
        x="Target",
        y="Value",
        title="Boxplots for variables in the training set \n demonstrating differences between teams meeting the target and not",
    )
    + pln.theme_bw()
    + pln.theme(figure_size=(25, 18), subplots_adjust={"wspace": 0.25})
)

Boxplots are not as informative as we would expect them to be. Probably because classes 0 and 1 share a lot of common values through tied games or games where one team won, but scoring only one goal. We need to then use more statistical methods to approach the problem - eyeballing is not enough.

---

We will run a series of statistical tests to see whether there are statistically significant differences for any of the variables.

In [ ]:
p_vals = []

for field in X_fields:
    _, p_val = scipy.stats.ttest_ind(
        X_train[field].loc[y_train == 0].values, X_train[field].loc[y_train == 1].values
    )
    p_vals.append(p_val)

In [ ]:
# run correction for multiple testing
is_significant, _, _, _ = statsmodels.stats.multitest.multipletests(
    p_vals, method="bonferroni"
)

In [ ]:
from IPython.display import Markdown as md

In [ ]:
md(
    """It appears, that there exists statistical difference for {} variables (after correction for multiple testing). 
We will plot density estimation for each of those.""".format(
        sum(is_significant)
    )
)

In [ ]:
plot_data = pd.concat(
    [X_train[np.array(X_fields)[is_significant]], y_train.astype("category")], axis=1
).melt(id_vars="target")

In [ ]:
(
    pln.ggplot(pln.aes(x="value"), plot_data)
    + pln.geom_density(pln.aes(fill="target", color="target"), alpha=0.2)
    + pln.facet_wrap("~variable", scales="free")
    + pln.labs(
        x="Density estimation",
        y="",
        title="Density estimation per target for statistically significant variables",
    )
    + pln.theme_bw()
    + pln.theme(figure_size=(20, 18), subplots_adjust={"wspace": 0.25, "hspace": 0.25})
)

Oddly enough, all the significant variables are related to the performance during **home games**. The shapes of the density estimators are quite similar, but for a number of cases non-target teams (so losing, tying, or winning, but scoring one goal) have greater density for lower-than-mean values for variables like `shots_on_target_avg_home`, `performance_acum_home` or `passes_avg_H`. It indicates that, on average, teams not meeting the target do have performance below average in important aspects like shots on target or number of passes, which correlates with the mental image of a "worse" team (technically worse - the team may still have won!).

## Models

We will build three models and assess their performance:
- **logistic regression** - A simple and quick to train and optimize model that will serve as a benchmark.
- **random forest** - Bagging model that is known to perform decently in multiple scenarios.
- **gradient boosting model** - We have enough data to train a boosting model - one of the most reliable models in terms of performance.

All of those models will have hyperparameters tuned (but not very deeply, to save time). We will be assessing their initial performance on 20 folds from repeated k-fold CV procedure.

In [ ]:
rcv = RepeatedKFold(
    n_splits=10, n_repeats=2
)  # we will be evaluating the model on 20 folds

In [ ]:
# define models
log_reg_model = Pipeline(
    (
        ("scaler", MinMaxScaler()),
        (
            "model",
            LogisticRegressionCV(
                cv=5, max_iter=500, n_jobs=2, scoring="roc_auc", refit=False
            ),
        ),
    )
)
# shallow grid search for brevity
rf_model = GridSearchCV(
    RandomForestClassifier(n_estimators=150),
    param_grid={"max_depth": [3, 6, 9, 12]},
    scoring="roc_auc",
    cv=5,
    refit=False,
    n_jobs=2,
)
# bayesian optimization of hyperparameters - otherwise LightGBM is kind of meh
lgbm_model = BayesSearchCV(
    lightgbm.LGBMClassifier(n_estimators=150),
    search_spaces={
        "max_depth": Integer(5, 15),
        "learning_rate": Real(1e-3, 0.5, prior="log-uniform"),
        "reg_alpha": Real(1e-3, 1, prior="log-uniform"),
        "reg_lambda": Real(1e-3, 1, prior="log-uniform"),
    },
    n_iter=25,
    cv=5,
    n_jobs=2,
)

In [ ]:
log_reg_results = cross_validate(
    log_reg_model, X_train, y_train, scoring="roc_auc", cv=rcv, n_jobs=2
)

In [ ]:
log_reg_results["test_score"]

In [ ]:
rf_results = cross_validate(
    log_reg_model, X_train, y_train, scoring="roc_auc", cv=rcv, n_jobs=2
)

In [ ]:
rf_results["test_score"]

In [ ]:
%%time
# be carefeul - it takes quite a long time to process!
lgbm_results = cross_validate(
    lgbm_model, X_train, y_train, scoring="roc_auc", cv=rcv, n_jobs=2
)

In [ ]:
lgbm_results["test_score"]

In [ ]:
plot_data = pd.concat(
    [
        pd.DataFrame({"model": "LR", "cv_scores": log_reg_results["test_score"]}),
        pd.DataFrame({"model": "RF", "cv_scores": rf_results["test_score"]}),
        pd.DataFrame({"model": "LGBM", "cv_scores": lgbm_results["test_score"]}),
    ]
)

(
    pln.ggplot(data=plot_data, mapping=pln.aes(x="model", y="cv_scores"))
    + pln.geom_boxplot()
    + pln.geom_hline(pln.aes(yintercept=0.5), size=2, linetype="dashed")
    + pln.labs(x="Model", y="ROC AUC", title="ROC AUC scores in 2x-repeated 10-fold CV")
    + pln.theme_bw()
)

As can be seen from the plot above, the performance of the model in thorough cross-validation is very poor, almost indistinguishable from a random model (with ROC AUC of 0.5). It may suggest that the variables that we use do not have sufficient explanatory power to deal with the problem at hand. But it's to early to assess, since the main culprit may be faulty problem formulation. Under the assumed approach, where each match makes up two observations (one for home team and one for away team), we may inject mixed-up information at the training stage. Our explanatory variables have, broadly speaking, 2 subgroups: a set of variables pertaining to the home team and the same variables for away team. But the target variable is not linked to them, since at times the home team has a value of 1 there and sometimes - the away team. The situation is exacerbated especially when there are two 0s for a given match.

What is more, the problem is ill-formulated also for another reason - two teams playing in one fixture are assessed independently, so it would be theoretically possible for both of them to receive a 1 as an output, which is clearly wrong, since two teams cannot win the same game... Should a model perform decently, that wouldn't be a problem, but given the weak performance in CV, we could be expecting all kind of undesired behaviors.

On a side note, we will not use the validation set that we set aside, as CV performance doesn't promise anything positive from such type of an experiment.

Therefore this approach may not be optimal and we need to reformulate the approach.

---

## Three-class approach

As an alternative to the (failing) approach above, we may propose a three-class approach, where a match can have three outcomes:
- home team wins scoring at least 2 goals;
- away team wins scoring at least 2 goals;
- no team wins scoring at least 2 goals (but still one of them may win 1-0).

This formulation can alleviate both of the issues we spotted above - there will be one observation per game, so only one outcome for the game will be possible (no contradictions) and the model will have a chance to tie home-team variables to the success (or not) of the home team, similarly with the away team.

### Prepare data for modeling

In [ ]:
# create a new variable that can take on three values - 1 if the home team meets requirements, 2 if the away team meets them and 0 otherwise
three_class_data = premiership_data.assign(
    three_class_target=lambda x: np.where(
        np.logical_and(x["goals_H"] > x["goals_A"], x["goals_H"] >= 2),
        1,
        np.where(np.logical_and(x["goals_A"] > x["goals_H"], x["goals_A"] >= 2), 2, 0),
    )
).drop(columns=["goals_H", "goals_A", "win_H", "win_A", "target_H", "target_A"])

Unlike in previous case, where we were using CV for initial validation of models, we will need now two validation sets: one to measure performance of the neural network during the training and one to make final conclusions. We will set aside the last two seasons for that reason. 19/20 for the network to indicate its performance and 20/21 as the final evaluation set.

In [ ]:
three_class_data["train_test_val"] = np.where(
    three_class_data["season"] == "19/20",
    "test",
    np.where(three_class_data["season"] == "20/21", "val", "train"),
)

In [ ]:
three_class_data["train_test_val"].value_counts()

The training set will still be big enough to train the network viably.

In [ ]:
X_train = three_class_data[X_fields].loc[three_class_data["train_test_val"] == "train"]
X_test = three_class_data[X_fields].loc[three_class_data["train_test_val"] == "test"]
X_val = three_class_data[X_fields].loc[three_class_data["train_test_val"] == "val"]

y_train = three_class_data["three_class_target"].loc[
    three_class_data["train_test_val"] == "train"
]
binarizer = LabelBinarizer()
y_train = binarizer.fit_transform(y_train)
y_test = binarizer.transform(
    three_class_data["three_class_target"].loc[
        three_class_data["train_test_val"] == "test"
    ]
)
y_val = binarizer.transform(
    three_class_data["three_class_target"].loc[
        three_class_data["train_test_val"] == "val"
    ]
)

In [ ]:
# we build a small feed-forward network with three outputs to address our
model = keras.Sequential()
model.add(keras.layers.Dense(128, input_shape=(len(X_fields),), activation="relu"))
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(3, activation="softmax"))

# train the model using SGD
adam = keras.optimizers.Adam(0.0001)
# categorical accuracy is not the best metric, but keras doesn't offer better ones off-the-shelf
model.compile(
    loss="categorical_crossentropy", optimizer=adam, metrics=["categorical_accuracy"]
)
# we declare early stopping criterion, but eventually - we do not use it
early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor="loss", patience=4, min_delta=1e-4
)

In [ ]:
nn_history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=150,
    batch_size=256,
    callbacks=[],
)

Let's see the model performance in the form of confusion matrix (on the test set, the same one that was used to control network's quality).

In [ ]:
y_test_preds = np.argmax(model.predict(X_test), axis=1)

In [ ]:
cm = confusion_matrix(
    three_class_data["three_class_target"]
    .loc[three_class_data["train_test_val"] == "test"]
    .values,
    y_test_preds,
)

In [ ]:
ConfusionMatrixDisplay(cm).plot()

In [ ]:
three_class_data["three_class_target"].loc[
    three_class_data["train_test_val"] == "train"
].value_counts().plot(kind="bar")

In [ ]:
print(
    classification_report(
        three_class_data["three_class_target"]
        .loc[three_class_data["train_test_val"] == "test"]
        .values,
        y_test_preds,
        target_names=[
            "Nothing interesting",
            "Home team triggered",
            "Away team trigerred",
        ],
    )
)

From the confusion matrix and classification report we can conclude that the network learned _something_, but it is not enough to establish a trustworthy model. The model is considerably skewed towards class 0 (none team met the requirements) and makes many mistakes trying to assign classifies matches to this group. On the other hand, class 2 (away team wins scoring 2 goals) is considerably underestimated. We see recall of only 0.1, so only one in ten matches ending with this outcome is actually recognized. But in general, there are very few matches classified as 2 by the model. This shouldn't be the case, since in the train set this is the least frequent class, but not to extend when it should be that heavily disregarded.

### Summary

Overall, it turned out that the task at hand is not a low-hanging fruit and seems not to be a task that is easy to model. There are some ways which one could take from now on:
- Train another model for the three-class problem (LightGBM, which tends to excel for tabular data). That could help in assessment of the poor performance of the network model as it could either be misspecification of the model or the data problem on its own.
- Carry out some feature engineering. One idea here could be to calculate differences in corresponding features between home and away teams. That would simplify the feature space. 
- Accept a little bit romantic approach that football at its core is unpredictable.